In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

fish_dataset_dir = '/kaggle/input/ai-unict-2024-challenge-1/'
fish_dataset_train_dir = '/kaggle/input/ai-unict-2024-challenge-1/train'
fish_dataset_test_dir = '/kaggle/input/ai-unict-2024-challenge-1/test'

# link to GoogleDrive folder:
gd_folder = "https://drive.google.com/drive/folders/1E7E0CDF98_MlKi8Mk8764cQwu004jjJZ?usp=sharing"

# link to GoogleDrive zip:
gd_zip = "https://drive.google.com/file/d/18O5LAK35yvTK3Ow-SZrU0p_Y-Yv3OOe0/view?usp=sharing"

In [ ]:
#Setup Gdrive file download extention 
!conda install -y gdown

In [ ]:
# Model1 weights
!gdown --id 1F6ib68f1HarOeMkoAG1Csdt9PgcS_Y5h

# Model2 weights
!gdown --id 1GO12RFjozr9V5sqcvApOaTbCR7cG9epH

# Model3 weights 1
!gdown --id 103KBMc5iLY4SQ7vyN4xK9FwpO7t-Kn71
# Model3 weights 2
!gdown --id 14dze-SSp5uMos_rdDY4e1ydTfs2Y7xGQ

# Model4 weights 1
!gdown --id 1odOpmPYR2YdYJAm121PnOQaRrgb3np3D

# Model4 weights 2
!gdown --id 1Lo-2zz9Ss5HJSF_kXn3U51Fd0QS1u5gt

 # **MODEL 1**

**We use a pre-trained ResNet18 and fine-tune the classifier on the small annotated dataset**

In [ ]:
class TestFishDataset(Dataset):
    def __init__(self, data_dir, transforms=T.Compose([])):
        self.data_dir = Path(data_dir)
        self.transforms = transforms
        self.files = sorted(os.listdir(self.data_dir))
        
    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        file_path = self.data_dir / self.files[i]
        img = Image.open(file_path).convert("RGB")
        img = self.transforms(img)
        return img,self.files[i]

In [ ]:
train_transforms = T.Compose([
        T.RandomHorizontalFlip(p=0.5),
        T.RandomRotation(degrees=30),
        T.Resize((224,224)),
        T.ToTensor(),
        T.Normalize(0.5, 0.5) 
    ])

test_transforms = T.Compose([
        T.Resize((224,224)),
        T.ToTensor(),
        T.Normalize(0.5, 0.5)
    ])

In [ ]:
def denormalize_and_resize(img, size=(128, 128)):
    img = img * 0.5 + 0.5  
    img = T.functional.resize(img, size)  
    return img

def imshow_single(img):
    img = img.numpy()  
    plt.imshow(np.transpose(img, (1, 2, 0)))  # (CxHxW --> HxWxC)
    plt.show()

In [ ]:
# Train set
train_set = datasets.ImageFolder(root=fish_dataset_train_dir, transform=train_transforms)
train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=4)

# Test set    
test_set = TestFishDataset(data_dir=fish_dataset_test_dir, transforms=test_transforms)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers =4)

In [ ]:
import random

print("Training set:\n")   
image, label = train_set[random.randint(0, 149)] 
image = denormalize_and_resize(image)
imshow_single(image)
print(f'Label: {label}')

print("\nTest set:\n")
image,names = test_set[random.randint(0, 147)]  
image = denormalize_and_resize(image)
imshow_single(image)

In [ ]:
import torch
import matplotlib.pyplot as plt

def train(net, loaders, optimizer, criterion, epochs=100, dev=torch.device('cpu')):
    best_val_loss = float('inf') 
    best_val_accuracy = float('-inf') 
    try:
        net = net.to(dev)
        
        # Initialize history
        history_loss = {"train": [], "val": []}
        history_accuracy = {"train": [], "val": []}
        
        # Process each epoch
        for epoch in range(epochs):
            
            # Initialize epoch variables
            sum_loss = {"train": 0, "val": 0}
            sum_accuracy = {"train": 0, "val": 0}
            
            # Process each split
            for split in ["train", "val"]:
                if split == "train":
                    net.train()
                else:
                    net.eval()
                    
                # Process each batch
                for (input, labels) in loaders[split]:
                   
                    input = input.to(dev)
                    labels = labels.to(dev)
                    # Reset gradients
                    optimizer.zero_grad()
                    # Compute output
                    if split == "train":
                        outputs = net(input)
                    else:
                        with torch.no_grad():
                            outputs = net(input)
                    loss = criterion(outputs, labels)
                    
                    # Update loss
                    sum_loss[split] += loss.item()
                    
                    # Check parameter update
                    if split == "train":
                        # Compute gradients
                        loss.backward()
                        # Optimize
                        optimizer.step()
                        
                    # Compute accuracy
                    preds = torch.argmax(outputs, 1) # 1 is the second dimension                    
                    batch_accuracy = (preds == labels).sum().item() / input.size(0)
                    
                    # Update accuracy
                    sum_accuracy[split] += batch_accuracy
                    
            # Compute epoch loss/accuracy
            epoch_loss = {split: sum_loss[split] / len(loaders[split]) for split in ["train", "val"]}
            epoch_accuracy = {split: sum_accuracy[split] / len(loaders[split]) for split in ["train", "val"]}

            # Model selection
            if epoch_loss["val"] < best_val_loss:
                torch.save({
                    "epoch": epoch,
                    "model_state_dict": net.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "val_loss": epoch_loss['val']
                }, './best_loss.pt')
                best_val_loss = epoch_loss['val']
                
            # Model selection    
            if epoch_accuracy["val"] > best_val_accuracy:
                torch.save({
                "epoch": epoch,
                "model_state_dict": net.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "val_accuracy": epoch_accuracy["val"]
                }, './best_accuracy.pt')
                best_val_accuracy = epoch_accuracy["val"]

            # Update history
            for split in ["train", "val"]:
                history_loss[split].append(epoch_loss[split])
                history_accuracy[split].append(epoch_accuracy[split])
                
            # Print info
            print(f"Epoch {epoch+1}:",
                  f"TrL={epoch_loss['train']:.4f},",
                  f"TrA={epoch_accuracy['train']:.4f},",
                  f"VL={epoch_loss['val']:.4f},",
                  f"VA={epoch_accuracy['val']:.4f}")
    except KeyboardInterrupt:
        print("Interrupted")
    finally:
        
        # Plot loss
        plt.title("Loss")
        for split in ["train", "val"]:
            plt.plot(history_loss[split], label=split)
        plt.legend()
        plt.show()
        
        # Plot accuracy
        plt.title("Accuracy")
        for split in ["train", "val"]:
            plt.plot(history_accuracy[split], label=split)
        plt.legend()
        plt.show()

In [ ]:
train_set = datasets.ImageFolder(root=fish_dataset_train_dir, transform=train_transforms)

val_idx = []
train_idx = []
counter = 0
idx = 0

for img,label in train_set:
    if counter < 2:
        val_idx.append(idx)
    else:
        train_idx.append(idx)
    idx += 1
    counter +=1
    
    if counter == 10:
        counter = 0


print ("validation set length: ",len(val_idx)) # 30
print ("\n validation set indices: \n",val_idx) # [0, 1, 10, 11, 20, 21, 30, 31, 40, 41, 50, 51, ... , 140, 141]
print ("\ntraing set length: ",len(train_idx)) # 120


from torch.utils.data import Subset

val_set = Subset(train_set, val_idx) 
train_set = Subset(train_set, train_idx)   

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_set,   batch_size=15, shuffle=False, num_workers=4)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers =4)

loaders = {
    "train": train_loader,
    "val": val_loader
}

In [ ]:
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn

my_resnet18 = resnet18(weights=ResNet18_Weights.DEFAULT) 
new_classifier = nn.Linear(my_resnet18.fc.in_features, 15)

my_resnet18.fc = new_classifier 

for param in my_resnet18.parameters():
    param.requires_grad = False

for param in my_resnet18.fc.parameters():
    param.requires_grad = True
    
import torch.optim as optim
resnet18_optimizer = optim.Adam(my_resnet18.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

dev = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(dev)

print("\nTraining...\n")
train(my_resnet18, loaders, resnet18_optimizer, criterion, epochs=30, dev=dev)

In [ ]:
def evaluate_and_save_predictions(net, test_loader, output_file="submission.csv", device=torch.device('cpu')):
    net = net.to(device)
    net.eval()
    
    predictions = []
    image_names = []

    with torch.no_grad():
        for inputs, img_names in test_loader:
            inputs = inputs.to(device)
            outputs = net(inputs)
            preds = torch.argmax(outputs, 1).cpu().numpy()
            predictions.extend(preds)
            image_names.extend(img_names)
    
    # Create a DataFrame and save to CSV
    df = pd.DataFrame({"image": image_names, "class": predictions})
    df.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")

In [ ]:
checkpoint = torch.load('/kaggle/working/Model1_best_accuracy.pt', map_location=torch.device(dev))
my_resnet18.load_state_dict(checkpoint['model_state_dict'])

evaluate_and_save_predictions(my_resnet18, test_loader)

# **MODEL 2**
**We train an autoencoder with a classifier on the large unannotated dataset and the small annotated dataset**

In [ ]:
import torch
import torch.nn as nn

class AutoencoderClassifier(nn.Module):
    def __init__(self, num_classes=15):
        super(AutoencoderClassifier, self).__init__()
        
       # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),  
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 224x224 => 112x112
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 112x112 => 56x56
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),  
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 56x56 => 28x28
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),  # 28x28 => 56x56
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # 56x56 => 112x112
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),  # 112x112 => 224x224
            nn.Sigmoid()
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(256 * 28 * 28, 512),  
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        flattened = encoded.view(encoded.size(0), -1)
        prediction = self.classifier(flattened)
        return decoded, prediction

In [ ]:
# Unannotated fish dataset
unlabeled_dir = "/kaggle/input/unannotated-dataset/images_all"

In [ ]:
class UnlabledDataset(Dataset):
    def __init__(self, data_dir, transforms=None):
        self.data_dir = Path(data_dir)
        self.transforms = transforms
        self.files = sorted(os.listdir(self.data_dir))
        
    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        file_path = self.data_dir / self.files[i]
        img = Image.open(file_path).convert("RGB")
        if self.transforms is not None:
            img = self.transforms(img)
        return img

In [ ]:
# unlabeled set 
unlabeled_set = UnlabledDataset(data_dir =unlabeled_dir, transforms=train_transforms)

unlabeled_train_idx = list(range(len(unlabeled_set))) 
import random
random.shuffle(unlabeled_train_idx)
unlabeled_num_train = len(unlabeled_set) - int(len(unlabeled_set) * 0.2) 
unlabeled_val_idx = unlabeled_train_idx[unlabeled_num_train:] 
unlabeled_train_idx = unlabeled_train_idx[:unlabeled_num_train] 

from torch.utils.data import Subset  

unlabeled_val_set = Subset(unlabeled_set, unlabeled_val_idx) 
unlabeled_set = Subset(unlabeled_set, unlabeled_train_idx)

unlabeled_loader = DataLoader(unlabeled_set, batch_size=128, shuffle=True,num_workers = 4)
unlabeled_val_loader = DataLoader(unlabeled_val_set,batch_size=128, shuffle=False, num_workers = 4)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers = 4)

unlabeled_loaders = {
    "train": unlabeled_loader,
    "val": unlabeled_val_loader
}

In [ ]:
def train_autoencoder_with_classifier(net,unlabeled_loaders,loaders,optimizer,criterion,criterion_autoencoder,epochs=100,dev=torch.device('cpu')):
    best_val_loss = float('inf')  
    best_val_accuracy = float('-inf')
    try:
        net = net.to(dev)
        # Initialize history
        history_loss_classification = {"train": [], "val": []}
        history_loss_reconstruction = {"train": [], "val": []}
        history_classification_accuracy = {"train": [], "val": []}

        # Process each epoch
        for epoch in range(epochs):
            sum_loss_classification = {"train": 0, "val": 0}
            sum_loss_reconstruction = {"train": 0, "val": 0}
            sum_accuracy_classification = {"train": 0, "val": 0}

            for split in ["train", "val"]:
                _ = net.train() if split == "train" else net.eval()
                
                for input in unlabeled_loaders[split]:
                    input = input.to(dev)
                    optimizer.zero_grad()
                    
                    if split == "train":
                        decoded_outputs, _ = net(input)
                    else:
                        with torch.no_grad():
                            decoded_outputs, _ = net(input)
                            
                    loss_autoencoder = criterion_autoencoder(decoded_outputs, input)
                    sum_loss_reconstruction[split] += loss_autoencoder.item()
            
                    if split == 'train':
                        loss_autoencoder.backward()
                        optimizer.step()

                for input, labels in loaders[split]:
                    input = input.to(dev)
                    labels = labels.to(dev)
                    optimizer.zero_grad()

                    if split == "train":
                        _,preds_outputs = net(input)
                    else:
                        with torch.no_grad():
                            _,preds_outputs = net(input)
                    
                    loss_classification = criterion(preds_outputs, labels)
                    sum_loss_classification[split] += loss_classification.item()
                    
                    if split == 'train':
                        loss_classification.backward()
                        optimizer.step()
                        
                    preds = torch.argmax(preds_outputs, 1)
                    batch_accuracy = (preds == labels).sum().item() / input.size(0)
                    sum_accuracy_classification[split] += batch_accuracy
             
            epoch_loss_classification = {split: sum_loss_classification[split] / len(loaders[split]) for split in ["train", "val"]}
            epoch_loss_reconstruction = {split: sum_loss_reconstruction[split] / len(unlabeled_loaders[split]) for split in ["train", "val"]}
            epoch_accuracy_classification = {split: sum_accuracy_classification[split] / len(loaders[split]) for split in ["train", "val"]}

            # Model selection
            if epoch_loss_classification["val"] < best_val_loss:
                torch.save({
                    "epoch": epoch,
                    "model_state_dict": net.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "val_loss": epoch_loss_classification['val']
                }, './best_loss.pt')
                best_val_loss = epoch_loss_classification['val']  
                
            # Model selection    
            if epoch_accuracy_classification["val"] > best_val_accuracy:
                torch.save({
                    "epoch": epoch,
                    "model_state_dict": net.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "val_accuracy": epoch_accuracy_classification["val"]
                }, './best_accuracy.pt')
                best_val_accuracy = epoch_accuracy_classification["val"]
            
            for split in ["train", "val"]:
                history_loss_classification[split].append(epoch_loss_classification[split])
                history_loss_reconstruction[split].append(epoch_loss_reconstruction[split])
                history_classification_accuracy[split].append(epoch_accuracy_classification[split])

            # Print classification info
            print(f"Epoch {epoch+1}:",
                  f"TrRL={epoch_loss_reconstruction['train']:.4f},",
                  f"RVL={epoch_loss_reconstruction['val']:.4f},",
                  f"TrL={epoch_loss_classification['train']:.4f},",
                  f"TrA={epoch_accuracy_classification['train']:.4f},",
                  f"VL={epoch_loss_classification['val']:.4f},",
                  f"VA={epoch_accuracy_classification['val']:.4f}"
                  
                 )
    except KeyboardInterrupt:
        print("Interrupted")
    finally:
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

        ax1.plot(history_loss_reconstruction["train"], label='Train Reconstruction')
        ax1.plot(history_loss_reconstruction["val"], label='Val Reconstruction')
        ax1.set_title("Reconstruction loss")
        ax1.set_xlabel("Epoch")
        ax1.set_ylabel("Loss")
        ax1.legend()

        ax2.plot(history_classification_accuracy["train"], label='Train Accuracy')
        ax2.plot(history_classification_accuracy["val"], label='Val Accuracy')
        ax2.set_title("Classification accuracy")
        ax2.set_xlabel("Epoch")
        ax2.set_ylabel("Accuracy")
        ax2.legend()

        ax3.plot(history_loss_classification["train"], label='Train Classification')
        ax3.plot(history_loss_classification["val"], label='Val Classification')
        ax3.set_title("Classification loss")
        ax3.set_xlabel("Epoch")
        ax3.set_ylabel("Loss")
        ax3.legend()

        plt.tight_layout() 
        plt.show()

In [ ]:
auto_encoder_with_classifier = AutoencoderClassifier()
optimizer_aec = torch.optim.Adam(auto_encoder_with_classifier.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
criterion_autoencoder = nn.MSELoss()
dev = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(dev)

# Train
print("\nTraining: ...\n")
train_autoencoder_with_classifier(auto_encoder_with_classifier, unlabeled_loaders, loaders, optimizer_aec, criterion, criterion_autoencoder, epochs=40, dev = dev)

In [ ]:
checkpoint = torch.load('/kaggle/working/Model2_best_accuracy.pt', map_location=torch.device(dev))
auto_encoder_with_classifier.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
import torch
import matplotlib.pyplot as plt

# we want to compare an original image sample with the reconstruced one
images, labels = next(iter(val_loader))
auto_encoder_with_classifier = auto_encoder_with_classifier.to('cpu')

rec_images, preds = auto_encoder_with_classifier(images)

images = denormalize_and_resize(images)
image_np = images.cpu().detach().numpy()
image_np = np.transpose(image_np, (0, 2, 3, 1))  

rec_images = denormalize_and_resize(rec_images)

rec_image_np = rec_images.cpu().detach().numpy()
rec_image_np = np.transpose(rec_image_np, (0, 2, 3, 1))  


print("Original Image:\n")
plt.imshow(image_np[0]) 
plt.show()
print("Original Image Label: ",labels[0].item())

print("\nReconstructed Image:")
plt.imshow(rec_image_np[0])  
plt.show()
print("\nReconstructed Image Label: ",torch.argmax(preds[0]).item())

In [ ]:
def autoencoder_evaluation_and_save_predictions(net, test_loader, output_file="submission.csv", device=torch.device('cpu')):
    net = net.to(device)
    net.eval()
    
    predictions = []
    image_names = []

    with torch.no_grad():
        for inputs, img_names in test_loader:
            inputs = inputs.to(device)
            _,outputs = net(inputs)
            preds = torch.argmax(outputs, 1).cpu().numpy()
            predictions.extend(preds)
            image_names.extend(img_names)
    
    # Create a DataFrame and save to CSV
    df = pd.DataFrame({"image": image_names, "class": predictions})
    df.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")

In [ ]:
autoencoder_evaluation_and_save_predictions(auto_encoder_with_classifier, test_loader)

# **MODEL 3**

**We created pseudo-labels with the autoencoder and performed classification with two training steps on ResNet50**

In [ ]:
# unlabeled set 
unlabeled_set = UnlabledDataset(data_dir =unlabeled_dir, transforms=test_transforms) # same test transforms
unlabeled_loader = DataLoader(unlabeled_set, batch_size = 64, shuffle =True, num_workers = 4)

In [ ]:
class PseudoLabeledDataset(Dataset):
    def __init__(self, data, transforms=T.Compose([])):
        self.data = data
        self.transforms = transforms
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_image, label = self.data[idx]
        input_image = self.transforms(input_image)
        return input_image, label

In [ ]:
from tqdm import tqdm

def aec_evaluate_and_get_pseudolabels(net, unlabeled_loader, device=torch.device('cpu')):
    net = net.to(device)
    net.eval()
    
    pseudo_labeled_list = []
    total_batches = len(unlabeled_loader)
    
    with torch.no_grad():
        for batch_idx, inputs in tqdm(enumerate(unlabeled_loader), total=total_batches, desc="Generating Pseudo-Labels"):
            inputs = inputs.to(device)
            _,outputs = net(inputs)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, 1).cpu().numpy()
            max_probs = torch.max(probs, dim=1)[0].cpu().numpy() 
            for input_image, prediction, confidence  in zip(inputs, preds, max_probs):
                if confidence >= 0.8:
                    pseudo_labeled_list.append((input_image.cpu(), prediction))
                

    return pseudo_labeled_list

In [ ]:
checkpoint = torch.load('/kaggle/working/Model2_best_accuracy.pt', map_location=torch.device(dev))
auto_encoder_with_classifier.load_state_dict(checkpoint['model_state_dict'])

pseudo_labeled_list = aec_evaluate_and_get_pseudolabels(auto_encoder_with_classifier, unlabeled_loader,device = dev)

In [ ]:
pseudo_labeled_transforms = T.Compose([
        T.RandomHorizontalFlip(p=0.5),
        T.RandomRotation(degrees=(45)),
        T.Resize((224,224)),
        T.Normalize(0.5, 0.5)
    ])

In [ ]:
from torch.utils.data import ConcatDataset
pseudo_labeled_set = PseudoLabeledDataset(pseudo_labeled_list, transforms = pseudo_labeled_transforms)
combined_set = ConcatDataset([train_set, pseudo_labeled_set])
print(len(pseudo_labeled_set))
print(len(combined_set))

In [ ]:
image, label = combined_set[(len(combined_set) - 1)] 
image = denormalize_and_resize(image)
imshow_single(image)
print(f'Label: {label}')

In [ ]:
combined_train_idx = list(range(len(combined_set))) 

import random
random.shuffle(combined_train_idx)
combined_num_train = len(combined_set) - int(len(combined_set) * 0.25) 
combined_val_idx = combined_train_idx[combined_num_train:] 
combined_train_idx = combined_train_idx[:combined_num_train] 

from torch.utils.data import Subset  

combined_val_set = Subset(combined_set, combined_val_idx) 
combined_set = Subset(combined_set, combined_train_idx)

combined_loader = DataLoader(combined_set, batch_size=64, shuffle=True,num_workers = 4)
combined_val_loader   = DataLoader(combined_val_set,   batch_size=64, shuffle=False, num_workers = 4)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers = 4)

combined_loaders = {
    "train": combined_loader,
    "val": combined_val_loader
}

**First, we trained the pre-trained ResNet50 on the combined dataset.**

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

my_resnet50 = resnet50(weights=ResNet50_Weights.DEFAULT)
my_resnet50.fc = nn.Linear(my_resnet50.fc.in_features, 15)  

resnet50_optimizer = optim.Adam(my_resnet50.parameters(), lr=0.00001)

print("\nTraining...\n")
train(my_resnet50, combined_loaders, resnet50_optimizer, criterion, epochs=25, dev=dev)

**Then we fine-tuned the classifier using the small annotated dataset.**

In [ ]:
checkpoint = torch.load('/kaggle/working/Model3_first_training_best_accuracy.pt', map_location=torch.device(dev))
my_resnet50.load_state_dict(checkpoint['model_state_dict'])

for param in my_resnet50.parameters():
    param.requires_grad = False

for param in my_resnet50.fc.parameters():
    param.requires_grad = True

resnet50_optimizer = optim.Adam(my_resnet50.parameters(), lr=0.01)

train(my_resnet50, loaders, resnet50_optimizer, criterion, epochs=30, dev=dev)

In [ ]:
checkpoint = torch.load('/kaggle/working/Model3_second_training_best_accuracy.pt', map_location=torch.device(dev))
my_resnet50.load_state_dict(checkpoint['model_state_dict'])

evaluate_and_save_predictions(my_resnet50, test_loader)

# **Model-4**
**We utilized pseudo-labels to perform classification with two training steps on ResNet18**

**First, we trained the pre-trained ResNet18 on the combined dataset.**

In [ ]:
my_resnet18 = resnet18(weights=ResNet18_Weights.DEFAULT) 
new_classifier = nn.Linear(my_resnet18.fc.in_features, 15)
my_resnet18.fc = new_classifier 

resnet18_optimizer = optim.Adam(my_resnet18.parameters(), lr=0.00001)

# Training on the combined set
print("\nTraining: ...\n")
train(my_resnet18, combined_loaders, resnet18_optimizer, criterion, epochs=25, dev=dev)

**Then we fine-tuned the classifier using the small annotated dataset.**

In [ ]:
checkpoint = torch.load('/kaggle/working/Model4_first_training_best_accuracy.pt', map_location=torch.device(dev))
my_resnet18.load_state_dict(checkpoint['model_state_dict'])

for param in my_resnet18.parameters():
    param.requires_grad = False

for param in my_resnet18.fc.parameters():
    param.requires_grad = True
    
resnet18_optimizer = optim.Adam(my_resnet18.parameters(), lr=0.01)

# Train
print("\nTraining: ...\n")
train(my_resnet18, loaders, resnet18_optimizer, criterion, epochs=30, dev=dev)

In [ ]:
checkpoint = torch.load('/kaggle/working/Model4_second_training_best_accuracy.pt', map_location=torch.device(dev))
my_resnet18.load_state_dict(checkpoint['model_state_dict'])
evaluate_and_save_predictions(my_resnet18, test_loader)